<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)



###### Lepidoptera = butterfies and moths
###### Hymenoptera = wasps, bees and ants
###### Hemiptera = true bugs (cicadas, aphids, shield bugs, ...)
###### Odonata = dragonflies
###### Diptera = fies
###### Araneae = spiders
###### Coleoptera = beetles

###### NOT IN DATASET
###### Orthoptera = grasshoppers

In [2]:
import os
import re
import sys
import json
import time
import numpy as np
import pprint as pp
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import concurrent.futures
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt


AUTO = tf.data.AUTOTUNE
pp = pp.PrettyPrinter()

!pip install --quiet tf-models-official==2.9.2
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')# load all the metadata

import tensorflow_models as tfm
from tensorflow_models.vision import box_ops as box

     |████████████████████████████████| 2.1 MB 26.9 MB/s 
     |████████████████████████████████| 238 kB 41.3 MB/s 
     |████████████████████████████████| 1.1 MB 53.3 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 118 kB 44.6 MB/s 
     |████████████████████████████████| 1.3 MB 38.0 MB/s 
     |████████████████████████████████| 352 kB 9.4 MB/s 
     |████████████████████████████████| 636 kB 50.0 MB/s 
     |████████████████████████████████| 4.6 MB 44.4 MB/s 


In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

# [Data Exploring](https://www.kaggle.com/code/mistag/starter-arthropod-taxonomy-orders-data-exploring)

The Arthropod Taxonomy Orders dataset is a collection of highres images annotated with labels from the taxanomy rank order. Annotations have been made with VoTT. VoTT stores all metadata in json files. In this kernel we will import all the metadata into DataFrames and export metadata to a variety of formats for object detection model training.
The dataset is distributed under CC BY-NC-SA 4.0

In [8]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
import glob

pfiles = glob.glob('/content/ArTaxOr/**/*.vott', recursive=True)

df = pd.DataFrame()
for p in pfiles:
  print(p)
  with open(p) as f:
    pdata = json.load(f)
    print(pdata)
    df = df.append(pd.DataFrame(list(pdata['assets'].values())), ignore_index=True)

df['path'] = df['path'].str.replace('file:F:/', '')

/content/ArTaxOr/Hymenoptera/annotations/Hymenoptera.vott
{'name': 'Hymenoptera', 'securityToken': 'Diptera Token', 'sourceConnection': {'name': 'Hymenoptera', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiMTg2ZWZmNzNmYzYzZmQ1MTRmZjdjMzNmNWZjN2EzMGY0NTlkNWE5NGVlM2IzYWI5OTEyMzFhNDgwYmIwYzVmYjQ4MTg0MzYzYjBhZWY0MzgxMTdkMDJlZTE3Zjc1MWI3IiwiaXYiOiJhMmUzODhiMjNkYjc3ZTRiOWIzOGU4ZmZhN2Q0ODE2ZmI0YTVjNzkxNjM2NmU5ZTYifQ=='}, 'id': 'gZN4ubW_-'}, 'targetConnection': {'name': 'Hymenoptera annotations', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiZDE5ZTkwMzg3YjJmYThjZGIxOGY3MTgzZjZkMjNjM2IxYzc4ODRjMzYxZDZmMGUxMzgzOGUzOGIwYzA2MGUwZmYzZjNkZWNhMjNhYTk4NzJkYWRhYmM2NzExZjM4MjkyMDkxOTIwZjQyY2M4MWIyZTE5Yzc3NDY1N2M3MjE5MDIiLCJpdiI6ImExY2Y0N2NjYTdiYzUxZDA4ZjY2OTEzOTUxYjNlODhkNmM2ZWZjY2M5ZjY4Zjk1ZCJ9'}, 'id': 'tjw012ylM'}, 'videoSettings': {'frameExtractionRate': 15}, 'tags': [{'name': 'Lepidoptera', 'color': '#5d

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
df

,format,id,name,path,size,state,type
0,jpg,18d6e4cfba5f7c11896155843770f541,000167b01850.jpg,ArTaxOr/Hymenoptera/000167b01850.jpg,"{'width': 2048, 'height': 1365}",2,1
1,jpg,d27c1ee10db64d7cefe6790d25336315,001aff3bb822.jpg,ArTaxOr/Hymenoptera/001aff3bb822.jpg,"{'width': 1257, 'height': 1257}",2,1
2,jpg,f0dc799b8a02ea34badcb02c441ca552,00747883359c.jpg,ArTaxOr/Hymenoptera/00747883359c.jpg,"{'width': 1493, 'height': 1494}",2,1
3,jpg,a08e2be7c5adcdb9fe4af4b61da822c7,007a168b796e.jpg,ArTaxOr/Hymenoptera/007a168b796e.jpg,"{'width': 2048, 'height': 1365}",2,1
4,jpg,562dd197a889d9bb5e735e156c17e323,01062c5763da.jpg,ArTaxOr/Hymenoptera/01062c5763da.jpg,"{'width': 1954, 'height': 1954}",2,1
...,...,...,...,...,...,...,...
15371,jpg,3bb331c49bebe70173dacf8886450f33,ffb51530da80.jpg,ArTaxOr/Diptera/ffb51530da80.jpg,"{'width': 2048, 'height': 1365}",2,1
15372,jpg,7df6d35809f39568168c934c3fea5e83,ffd97e78e0f4.jpg,ArTaxOr/Diptera/ffd97e78e0f4.jpg,"{'width': 2164, 'height': 1624}",2,1
15373,jpg,bc20cefe79fb9f20892173fd9e83ac68,ffe64903618e.jpg,ArTaxOr/Diptera/ffe64903618e.jpg,"{'width': 1712, 'height': 1590}",2,1
15374,jpg,b14ab6c907afb07047e3aee18411686a,fff86b0ae807.jpg,ArTaxOr/Diptera/fff86b0ae807.jpg,"{'width': 3815, 'height': 3518}",2,1


In [11]:
labels = pd.DataFrame(list(pdata['tags']))[:-3]
labels

,name,color
0,Lepidoptera,#5db300
1,Coleoptera,#e81123
2,Hymenoptera,#6917aa
3,Diptera,#015cda
4,Araneae,#4894fe
5,Hemiptera,#257ffe
6,Odonata,#257ffe


In [12]:
p = 'ArTaxOr/Diptera/fff86b0ae807.jpg'
index = p[::-1].find('/')+1
p[:-index]

'ArTaxOr/Diptera'

In [13]:
anno=pd.DataFrame(columns=['label', 'label_idx', 'xres', 'yres', 'height', 'width', 'left', 'top', 
                           'right', 'bottom', 'area', 'xcenter', 'ycenter', 'blurred',
                           'occluded', 'truncated', 'file', 'id'])


#get all the image's annotation details (object data) from a json file and store it in a dataframe
for i in range(len(df)):  #15376 images(with details)
  p = df['path'][i]
  index = p[::-1].find('/')+1
  json_file=f'/content/{p[:-index]}/annotations/{df["id"][i]}-asset.json'

  if os.path.isfile(json_file):
    with open(json_file) as f:
      adata = json.load(f)
    xres=adata['asset']['size']['width']
    yres=adata['asset']['size']['height'] 
    for j in range(len(adata['regions'])):
            h=adata['regions'][j]['boundingBox']['height']/yres
            w=adata['regions'][j]['boundingBox']['width']/xres
            tags=adata['regions'][j]['tags']
            anno=anno.append({'label': tags[0],
                              'label_idx': labels[labels.name==tags[0]].index[0],
                              'xres': xres,
                              'yres': yres,
                              'height': h,
                              'width': w,                              
                              'left': adata['regions'][j]['boundingBox']['left']/xres,
                              'top': adata['regions'][j]['boundingBox']['top']/yres,
                              'right': adata['regions'][j]['boundingBox']['left']/xres+w,
                              'bottom': adata['regions'][j]['boundingBox']['top']/yres+h, 
                              'area': h*w,
                              'xcenter': adata['regions'][j]['boundingBox']['left']/xres+0.5*w,
                              'ycenter': adata['regions'][j]['boundingBox']['top']/yres+0.5*h,
                              'blurred': int(any(ele == '_blurred' for ele in tags)),
                              'occluded': int(any(ele == '_occluded' for ele in tags)),
                              'truncated': int(any(ele == '_truncated' for ele in tags)),
                              'file': adata['asset']['path'].replace('file:F:/',''),
                              'id': adata['asset']['id'],}, ignore_index=True)

In [14]:
anno.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
17643,Diptera,3,1821,1847,0.761494,0.510204,0.255740,0.134100,0.765944,0.895594,0.388517,0.510842,0.514847,1,0,0,ArTaxOr/Diptera/38256b1a8db4.jpg,ff1a5b4866bf89799dec2b5169e47bf0
9638,Hemiptera,5,2048,1639,0.797893,0.892720,0.057675,0.120690,0.950395,0.918582,0.712295,0.504035,0.519636,0,0,0,ArTaxOr/Hemiptera/3e6e93701da0.jpg,d955fca00be4e74f61496faba1356674
7959,Lepidoptera,0,1920,1285,0.477858,0.203997,0.448656,0.216851,0.652653,0.694709,0.097482,0.550655,0.455780,0,0,0,ArTaxOr/Lepidoptera/b35a0002e026.jpg,b898f98161fecb3860dd831464d46c92
15684,Odonata,6,2048,1712,0.371495,0.655599,0.145833,0.329439,0.801432,0.700935,0.243552,0.473633,0.515187,0,0,0,ArTaxOr/Odonata/68c53348f30f.jpg,f30297c9a0056e754a7ea149e57a1d1e
1166,Hymenoptera,2,4272,2848,0.122027,0.117850,0.000000,0.782494,0.117850,0.904521,0.014381,0.058925,0.843508,0,0,1,ArTaxOr/Hymenoptera/6417716bc248.jpg,3c95961633433048a5b2196390899d23


In [ ]:
sns.relplot(x="width", y="height", hue="label", col="label", data=anno)

In [ ]:
sns.jointplot(x='width', y='height', data=anno.loc[anno['label'] == 'Diptera'])

In [ ]:
sns.set(rc={'figure.figsize': (12,6)})
sns.violinplot(x=anno['label'], y=anno['area'])

In [ ]:
graph=sns.countplot(data=anno, x='label')
graph.set_xticklabels(graph.get_xticklabels(), rotation=90)

for p in graph.patches:
  height = p.get_height()
  graph.text(p.get_x() + p.get_width()/2, height + 0.1, height, ha='center')

In [ ]:
temp = 'truncated'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
sns.countplot(x=temp, hue='label', data=df2)

In [ ]:
temp = 'blurred'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
sns.countplot(x=temp, hue='label', data=df2)

In [ ]:
temp = 'occluded'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
sns.countplot(x=temp, hue='label', data=df2)

## Image exploration


In [41]:
def attribution(fname):
    img = Image.open(fname)
    exif_data = img._getexif()
    img.close()
    if len(exif_data[315]) > 0:
        s='Photo: '+exif_data[315]
    else:
        s=exif_data[37510][8:].decode('ascii')
    return s

def plot_img(axes, image_df, highlight=True):
    image_file_name='/content/'+image_df.iloc[0].file
    im = Image.open(image_file_name)
    im.thumbnail((300,300),Image.ANTIALIAS)  #A small image representation of a larger image
    draw = ImageDraw.Draw(im)
    xres, yres = im.size[0], im.size[1] # X, Y Resolution
    for i in range(len(image_df)):
        selected_img = image_df.iloc[i]
        if highlight==True:
            color=(255, 0, 0) if i == 0 else (128, 128, 128)          
        else:
            color=labels[labels.name == selected_img.label].color.iloc[0]
        
        #draw a boundingbox
        draw.rectangle([int(selected_img['left']*xres),
                        int(selected_img['top']*yres),
                        int(selected_img['right']*xres),
                        int(selected_img['bottom']*yres)], outline=color, width=2)
        
    plt.setp(axes, xticks=[], yticks=[])  #remove the grids
    axes.set_title(image_df.iloc[0].label+'\n'+attribution(image_file_name))
    plt.imshow(im)

In [ ]:
fig = plt.figure(figsize=(16,26))
for i in range(len(labels)):
  for j in range(3):
    #Select 3 images with the largest area
    # ldf=anno[anno.label == labels.name[i]].nlargest(3, 'area') 

    #Select images with the most objects
    a=anno[anno.label == labels.name[i]]['id'].value_counts()
    ldf=anno[anno.id == a.index[j]]
    axes = fig.add_subplot(len(labels), 3, 1+i*3+j)
    plot_img(axes, anno[anno.id == ldf.iloc[j].id].sort_values(by=['area'], ascending=False), highlight=False)

## Metadata export

export metadata to various formats for object detection model training


### CSV

In [24]:
header = ['file', 'label', 'height', 'width', 'left', 'top', 'right', 'bottom'] # change as required
anno.to_csv('./ArTaxOr.csv', index=False, columns = header) 

### Pascal VOC

Pascal VOC files are xml format, and there is one xml file per image file, with same name.

In [25]:
!{sys.executable} -m pip install pascal_voc_writer
from pascal_voc_writer import Writer

if not os.path.exists('voc'):
    os.mkdir('voc')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/')
    image_xml = p[2].replace('.jpg','.xml') 
    width, height = ldf.xres[0], ldf.yres[0]
    
    writer = Writer(df.path[i], width, height)
    for j in range(len(ldf)):
        writer.addObject(ldf.label[j], 
                         int(ldf.left[j]*width), 
                         int(ldf.top[j]*height), 
                         int(ldf.right[j]*width),
                         int(ldf.bottom[j]*height))
    writer.save(f'./voc/{image_xml}')
print(os.listdir("./voc"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['1e492b843c41.xml', '3861aa18d649.xml', 'dc639933d695.xml', '0e1b7d9de0cd.xml', 'd30b56660828.xml', 'be8ad54c16e6.xml', '7fbeb3d69647.xml', '23f5367ad981.xml', '670634a7526e.xml', '36f364e5fd60.xml', '2934e8b0dcf0.xml', '9af64cb77541.xml', 'bf837ce3425b.xml', '57b25747937b.xml', '5e1160f0e027.xml', 'd661af7b08cb.xml', '001aff3bb822.xml', '7a335b8d57e3.xml', 'e1dc6d5c544a.xml', '4bcedddb3594.xml', 'b28f2817e615.xml', '0da5d1a40760.xml', 'cec3a55cfdf2.xml', '442aedf0126d.xml', 'b1ae37942954.xml', 'c6785ef7c4bb.xml', 'fa65dbcd8622.xml', '4984d11dbcd9.xml', 'b3f5de087069.xml', '7ce64a9cd3d2.xml', '376202a7f261.xml', 'b6d2b1ea0cba.xml', 'e47642820251.xml', '9fb0ce42b856.xml', '1ca55d7860b2.xml', 'd0cfc9568203.xml', 'f6ea54a321cc.xml', '4acb0ec57c45.xml', 'ff659e7b4d90.xml', 'd0003732821a.xml', '3b60727f6de8.xml', '7c0111849c0b.xml', 'e0fad1dae2b3.xml', '55887a3970bc.xml', '6bcdf9e76b7f.xml', 

### Darknet YOLOv3
Darknet expects one annotation file per image file. Each object is described by:

`class x_center y_center width height`

In [26]:
if not os.path.exists('labels'):
    os.mkdir('labels')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/') 
    image_txt = p[2].replace('.jpg','.txt') 
    
    file=open(f'./labels/{image_txt}','w')
    for j in range(len(ldf)):
        label_name=labels[labels.name == ldf.label[j]].index.to_list()
        file.write(f'{label_name[0]} {ldf.xcenter[j]} {ldf.ycenter[j]} {ldf.width[j]} {ldf.height[j]}\n')
    file.close()
print(os.listdir("./labels"))

['6991bb71724f.txt', 'fbdc9f9759fb.txt', 'b54441af82f6.txt', 'cb9ddda3daf6.txt', 'd1f3abf1e0be.txt', '6c0220f4597b.txt', 'ab18c0a0dbf1.txt', '1b07ff689641.txt', 'ce3119fa042a.txt', '5786ad212436.txt', '8d1e1166fd0d.txt', '5a61c7a644bf.txt', '39eedc3a4314.txt', '4dceab058396.txt', '1d77e58fc9a4.txt', '65bef96e00c5.txt', '3d1f44e2c2ae.txt', '1a7094e938d0.txt', '54eb1497aeb3.txt', 'b08eab95abd8.txt', 'a500ca109214.txt', '0afb88d88ac2.txt', '0cce38ad8be7.txt', '2c82a908b8e2.txt', 'e69c945a024c.txt', '5ccaa221bdfc.txt', '547a0f4856ec.txt', '5f81aed05f15.txt', '492f794a456b.txt', '4caf47fe0429.txt', '5fdcf59b4eda.txt', '86e4ce1b7864.txt', 'ada62cde4619.txt', 'f11459e61f35.txt', 'c726e2e09e39.txt', '3823657605be.txt', '89c7e2fb835f.txt', '779934751f1f.txt', '3b3dd1d57225.txt', '47272031405e.txt', '15d273cddbc1.txt', '761c71b68bc4.txt', 'a615d17199c3.txt', '4f1018fc18a5.txt', '1f1e907cb664.txt', '7d04c44e8414.txt', '79a3c61d43ea.txt', '8b75639d9269.txt', '2b859607f755.txt', '62368a715b0d.txt',

### Pickle

In [27]:
labels.to_pickle('./ArTaxOr_labels.pkl')
df.to_pickle('./ArTaxOr_filelist.pkl')
anno.to_pickle('./ArTaxOr_objects.pkl')

### TensorFlow TFRecords

#### Protocol buffers
TFRecords are Protocol buffers. Protocol buffers are a way of serializing structured data in a compact and efficient way. A Protocol buffer message is defined by a .proto file. Example:

```
syntax = "proto3"; // define protocol version

// Define "Employee" message with 3 fields
message Employee {
 string name = 1; // a unique number is assigned to each field
 int32 company_id = 2;
 string address = 3;
}

```


A single .proto-file can define multiple messages and messages can be nested. The beauty of protocol buffers is that the .proto-files can be compiled into a library (language of choice) with access functions using the protoc compiler. But for TFRecords the access functions we need are already part of TensorFlow, so no need to worry about compiling .proto files.

#### TFRecord format with simple data
TFRecords are made to support just about any type of data, and that means nesting basic features into a hierarchy of features.TFRecords supports only three different data types:

bytes
float
int64

So any data must be converted into one of these three types.

In [28]:
# tf.train.Feature can be used to convert data into lists of these types.
int_list = tf.train.Feature(int64_list=tf.train.Int64List(value=np.arange(8)))
int_list

int64_list {
  value: 0
  value: 1
  value: 2
  value: 3
  value: 4
  value: 5
  value: 6
  value: 7
}

In [29]:
float_list = tf.train.Feature(float_list=tf.train.FloatList(value=np.random.rand(10)))
float_list

float_list {
  value: 0.4250499904155731
  value: 0.15113630890846252
  value: 0.24063925445079803
  value: 0.37844735383987427
  value: 0.43516477942466736
  value: 0.8956334590911865
  value: 0.039077915251255035
  value: 0.6202369332313538
  value: 0.25040799379348755
  value: 0.7084002494812012
}

In [30]:
bytes_list = tf.train.Feature(bytes_list=tf.train.BytesList(value=['hello, my name is Esmail'.encode()]))
bytes_list

bytes_list {
  value: "hello, my name is Esmail"
}

In [31]:
features = tf.train.Features(feature={
    'integers': int_list,
    'float': float_list,
    'description': bytes_list
})
features

feature {
  key: "description"
  value {
    bytes_list {
      value: "hello, my name is Esmail"
    }
  }
}
feature {
  key: "float"
  value {
    float_list {
      value: 0.4250499904155731
      value: 0.15113630890846252
      value: 0.24063925445079803
      value: 0.37844735383987427
      value: 0.43516477942466736
      value: 0.8956334590911865
      value: 0.039077915251255035
      value: 0.6202369332313538
      value: 0.25040799379348755
      value: 0.7084002494812012
    }
  }
}
feature {
  key: "integers"
  value {
    int64_list {
      value: 0
      value: 1
      value: 2
      value: 3
      value: 4
      value: 5
      value: 6
      value: 7
    }
  }
}

The last step is to create an example TFRecord. So, example is not a very good name, but that is what the TFRecord is called. We simply pass the list of features to `tf.train.Example.`

In [32]:
tf_record = tf.train.Example(features=features)
tf_record

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.4250499904155731
        value: 0.15113630890846252
        value: 0.24063925445079803
        value: 0.37844735383987427
        value: 0.43516477942466736
        value: 0.8956334590911865
        value: 0.039077915251255035
        value: 0.6202369332313538
        value: 0.25040799379348755
        value: 0.7084002494812012
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

 write this record to a file using tf.io.TFRecordWriter

In [33]:
fname='example1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tf_record.SerializeToString())
print(f"Size of {fname} is {os.path.getsize(fname)} bytes")

Size of example1.tfrecord is 144 bytes


Next, read the file back in and access the data using tf.data.TFRecordDataset and tf.train.Example.FromString:

In [34]:
dataset = tf.data.TFRecordDataset(fname)
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
parsed

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.4250499904155731
        value: 0.15113630890846252
        value: 0.24063925445079803
        value: 0.37844735383987427
        value: 0.43516477942466736
        value: 0.8956334590911865
        value: 0.039077915251255035
        value: 0.6202369332313538
        value: 0.25040799379348755
        value: 0.7084002494812012
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

Access each feature using `parsed.features.feature['<feature name>']`



In [35]:
parsed.features.feature['description'].bytes_list.value[0].decode()

'hello, my name is Esmail'

In [36]:
parsed.features.feature['integers'].int64_list.value[:]

[0, 1, 2, 3, 4, 5, 6, 7]

In [37]:
parsed.features.feature['float'].float_list.value[:]

[0.4250499904155731,
 0.15113630890846252,
 0.24063925445079803,
 0.37844735383987427,
 0.43516477942466736,
 0.8956334590911865,
 0.039077915251255035,
 0.6202369332313538,
 0.25040799379348755,
 0.7084002494812012]

####TFRecords with image data

To make input function efficient, all preprocessing steps should be performed when creating TFRecords, otherwise the input function must repeat preprocessing each time the training passed over the TFRecord. Typical preprocessing steps are:

##### Decode image from jpg etc.
##### Resize image
##### Convert to float
##### Normalize to [0,1] range

Augumentation is not something to do in a TFRecord, since agumentation could be different each time training passes over the data

In [40]:
import hashlib
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
ARTAXOR_PATH = '/content/ArTaxOr/'

pickles='/content/'
objectdf=pd.read_pickle(pickles+'ArTaxOr_objects.pkl')
labels=pd.read_pickle(pickles+'ArTaxOr_labels.pkl')
objectdf.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
943,Hymenoptera,2,2048,1366,0.603306,0.374914,0.070296,0.287093,0.445210,0.890399,0.226188,0.257753,0.588746,0,0,0,ArTaxOr/Hymenoptera/53d3f440ad9f.jpg,dd9fd48b1ceec9f86503f6e57e6c43b8
1483,Hymenoptera,2,2048,1365,0.154085,0.175741,0.241902,0.627262,0.417643,0.781347,0.027079,0.329773,0.704305,0,0,0,ArTaxOr/Hymenoptera/7dbb6b74325d.jpg,e8a49d718c0608c4ae9a3ca507396779
10071,Hemiptera,5,2048,1536,0.181992,0.278736,0.346983,0.363985,0.625718,0.545977,0.050728,0.486351,0.454981,0,0,0,ArTaxOr/Hemiptera/5ff4b71c660a.jpg,aec9da598c9634c465023229689e16e4
18168,Diptera,3,1920,1440,0.181034,0.121408,0.822917,0.678161,0.944325,0.859195,0.021979,0.883621,0.768678,0,1,0,ArTaxOr/Diptera/6c7cfc59967e.jpg,2acfecfbd1fc0e1052d618c08816507c
8038,Lepidoptera,0,1536,2048,0.666667,0.513410,0.217114,0.162835,0.730524,0.829502,0.342273,0.473819,0.496169,0,0,0,ArTaxOr/Lepidoptera/bb60bc279ece.jpg,7a7c615a7f73743a6d449420bafc5a98


Define a function that creates a tf.train.Example from an image and the objects contained within.

    Note that TensorFlow Object Detection API expects jpeg-encoded image data

, so the only preprocessing to be done is resize (optional) before the image is re-encoded (via a memory buffer using BytesIO).


The input to this function is a DataFrame that contains one row per object and the columns shown above. This record is generous with features, but that is to make sure that the TF Object Detection API will successfully run the evaluation phase during training.

In [43]:
# Fetch attribution string from image EXIF data
def get_attribution(file):
    with Image.open(file) as img:
        exif_data = img._getexif()
    s='Photo: unknown'
    if exif_data is not None:
        if 37510 in exif_data:
            if len(exif_data[37510]) > 0:
                s = exif_data[37510][8:].decode('ascii')
        if 315 in exif_data:
            if len(exif_data[315]) > 0:
                s = 'Photo: ' + exif_data[315]
    return s

# Create example for TensorFlow Object Detection API
def create_tf_example(imagedf, longest_edge=1024):  
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]
    filename=fname.split('/')[-1] # exclude path
    by = get_attribution(fname)
    img = Image.open(fname, "r")
    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)
    height = img.size[1] # Image height
    width = img.size[0] # Image width
    buf= BytesIO()
    img.save(buf, format= 'JPEG') # encode to jpeg in memory
    encoded_image_data= buf.getvalue()
    image_format = b'jpeg'
    source_id = filename.split('.')[0]
    license = 'CC BY-NC-SA 4.0'
    # A hash of the image is used in some frameworks
    key = hashlib.sha256(encoded_image_data).hexdigest()   
    # object bounding boxes 
    xmins = imagedf.left.values # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = imagedf.right.values # List of normalized right x coordinates in bounding box
    ymins = imagedf.top.values # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = imagedf.bottom.values # List of normalized bottom y coordinates in bounding box
    # List of string class name & id of bounding box (1 per box)
    object_cnt = len(imagedf)
    classes_text = []
    classes = []
    for i in range(object_cnt):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1+imagedf.label_idx.iloc[i])
    # unused features from Open Image 
    depiction = np.zeros(object_cnt, dtype=int)
    group_of = np.zeros(object_cnt, dtype=int)
    occluded = imagedf.occluded.values #also Pascal VOC
    truncated = imagedf.truncated.values # also Pascal VOC
    # Pascal VOC
    view_text = []
    for i in range(object_cnt):
        view_text.append('frontal'.encode())
    difficult = np.zeros(object_cnt, dtype=int)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/license': tf.train.Feature(bytes_list=tf.train.BytesList(value=[license.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode()])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'image/object/depiction': tf.train.Feature(int64_list=tf.train.Int64List(value=depiction)),
        'image/object/group_of': tf.train.Feature(int64_list=tf.train.Int64List(value=group_of)),
        'image/object/occluded': tf.train.Feature(int64_list=tf.train.Int64List(value=occluded)),
        'image/object/truncated': tf.train.Feature(int64_list=tf.train.Int64List(value=truncated)),
        'image/object/difficult': tf.train.Feature(int64_list=tf.train.Int64List(value=difficult)),
        'image/object/view': tf.train.Feature(bytes_list=tf.train.BytesList(value=view_text))
    }))
    return tf_record